In [44]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import time
import random

In [45]:
def abrirImagenesEscaladas( carpeta, escala=32 ):
    # abre todas las imagenes de la carpeta, y las escala de tal forma que midan (escala x escala)px
    # devuelve las imagenes aplanadas -> vectores de tamano escala^2 con valores entre 0 y 1
    imagenes = []

    for dirpath, dirnames, filenames in os.walk(carpeta):
        for file in filenames:
            file_path = os.path.join(dirpath, file)
            if os.path.isfile(file_path):
                img = Image.open(file_path)
                img = img.resize((escala, escala))
                img = img.convert('L')
                img = np.asarray(img)
                img = img.reshape((escala**2)) / 255.0
                imagenes.append(img)
    return imagenes

In [46]:
from derivada_w3 import fAndG as fAndG_w
from derivada_b2 import fAndG as fAndG_b

In [47]:
def balancear_datos():
    imagenes_train_normal = abrirImagenesEscaladas("chest_xray/train/NORMAL")
    imagenes_train_pneumonia = abrirImagenesEscaladas("chest_xray/train/PNEUMONIA")

    cantidad_imagenes = min(len(imagenes_train_normal), len(imagenes_train_pneumonia))
    imagenes_train_normal = imagenes_train_normal[:cantidad_imagenes]
    imagenes_train_pneumonia = imagenes_train_pneumonia[:cantidad_imagenes]

    return [imagenes_train_normal, imagenes_train_pneumonia]

In [48]:
imagenes = balancear_datos()

In [ ]:
def L_w(i,w,b,d):
    t0=np.tanh(np.dot(w,i)+b)
    return (1-t0**2)*(((1+t0)/2)-d)*i

In [ ]:
def L_b(i,w,b,d):
    t0=np.tanh(np.dot(w,i)+b)
    return (1-t0**2)*(((1+t0)/2)-d)

In [ ]:
def dL_dw(w, b, imagenes, diagnosticos):
    gradiente = np.zeros_like(w)
    for i, imagen in enumerate(imagenes):
        gradiente += L_w(imagen, w, b, diagnosticos[i])
    return gradiente / len(imagenes)

In [ ]:
def dL_db(w, b, imagenes, diagnosticos):
    gradiente = 0
    for i, imagen in enumerate(imagenes):
        gradiente += L_b(imagen, w, b, diagnosticos[i])
    return gradiente / len(imagenes)

In [ ]:
def f(i, w, b):
    tan=np.tanh(np.dot(w, i) + b)
    return (tan+1)/2

In [ ]:
def func_L(w, b, imagenes, diagnosticos):
    error = 0
    for i, imagen in enumerate(imagenes):
        prediccion = f(imagen, w, b)
        error += (prediccion - diagnosticos[i])**2
    return error / len(imagenes)

In [ ]:
def descenso_grad(w, b, grad_w, grad_b, alpha):
    w = w - alpha* grad_w
    b = b - alpha* grad_b
    return w, b

In [ ]:
def descenso_por_gradiente(imagenes, alpha):
    w_t = np.random.rand(imagenes[0][0].shape[0])
    b_t = np.random.rand()

    iteracion = 0
    TOLERANCIA=1e-6

    while iteracion < 1000:
        print("Iteración: ", iteracion, "- Mínimo alcanzado hasta el momento: ", L(imagenes, w_t, b_t))

        w_t_mas_1 = w_t - alpha * dL_dw(imagenes, w_t, b_t)
        b_t_mas_1 = b_t - alpha * dL_db(imagenes, w_t, b_t)

        if abs(func_L(imagenes, w_t_mas_1, b_t_mas_1) - func_L(imagenes, w_t, b_t)) < TOLERANCIA:
            break
    
        w_t = w_t_mas_1
        b_t = b_t_mas_1
        iteracion += 1

    return [w_t, b_t]